<a href="https://colab.research.google.com/github/nielsvanesseveldt/Sociophysics2/blob/main/3boxes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import time
import pandas as pd
from datetime import datetime
from IPython.display import HTML
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
import scipy.stats as sci
from scipy.stats import entropy

%pylab inline
!pip install sociophysicsdatahandler
from sociophysicsDataHandler import SociophysicsDataHandler

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['sci', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


  Created wheel for sociophysicsdatahandler: filename=sociophysicsDataHandler-1.2.5-py3-none-any.whl size=7189 sha256=c8b5d1d0d2f01858af80304b3fdebd297089d34bd6a25ed63d6fe73dba6a2ff2
  Stored in directory: /root/.cache/pip/wheels/35/ce/d9/523386d14f4f73ec476157dc96a8d14c35f92cc8362716bb9e
  Created wheel for pyocclient: filename=pyocclient-0.6-py3-none-any.whl size=26057 sha256=64c2a3bb9a036cb81b8b7cea0c49b1eb56fdfe46efab18961310536635b94743
  Stored in directory: /root/.cache/pip/wheels/fd/4b/94/c7ed8749c157aeef88f1a23dfe0709c3b64b5ec36964d3d15f
Successfully built sociophysicsdatahandler pyocclient


In [3]:
!cat auth.txt

guest
YDPGL-XREXC-QGMZS-UAKES

In [4]:
#Definition of the 3 boxes
xmin = -5134
ymin = -17731
xmax =  75135
ymax =  11645

area_1 = 15
area_2 = 10
area_3 = 8

leftbound_1 = xmin
rightbound_1 = 31789
leftbound_2 = rightbound_1
rightbound_2 = 62610
leftbound_3 = rightbound_2
rightbound_3 = xmax

In [5]:
dh = SociophysicsDataHandler()

UTCs = pd.DataFrame()
UTCs.loc[0,'UTC'] =  1644818490000
UTCs.loc[1,'UTC'] =  1644818730000 
UTCs.loc[2,'UTC'] =  1644819150000
UTCs.loc[3,'UTC'] =  1644820290000
UTCs.loc[4,'UTC'] =  1644820590000
UTCs.loc[5,'UTC'] =  1644820950000

#dens0 = dataframe of densities in the 3 boxes for the first train
dens0 = pd.DataFrame(columns = (1, 2, 3))
dens1 = pd.DataFrame(columns = (1, 2, 3))
dens2 = pd.DataFrame(columns = (1, 2, 3))
dens3 = pd.DataFrame(columns = (1, 2, 3))
dens4 = pd.DataFrame(columns = (1, 2, 3))
dens5 = pd.DataFrame(columns = (1, 2, 3))

UTCs['UTC'] = UTCs['UTC'] - 30000


In [9]:
day = 24
file_path = 'ehv/platform2.1/202202%s/' %(day)
flst = dh.list_files(file_path)
hour = 8
# Fetch the data, [6] = 8:00-8:59 NO THIS IS THE RIGHT ONE: 6:00-6:59, [7] = 9:00-9:59 etc.
dh.fetch_prorail_data_from_path(file_path + '/' + flst.name[hour])

pd.set_option('float_format', '{:f}'.format)

train0 = dh.df[dh.df['date_time_utc']== 1845820950000]
file_path

targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220224/


HTTPResponseError: ignored

In [12]:
from pandas.core.arrays import integer
# Select Eindhoven platform and the date:
i = 0
day = 13
while i < 10:
  
  if i%2 == 0:
    day = day + 1
    hour = 7
  else:
    hour = 8

  file_path = 'ehv/platform2.1/202202%s/' %(day)
  print(day)
  fLst = dh.list_files(file_path)

  # Fetch the data, [6] = 8:00-8:59 NO THIS IS THE RIGHT ONE: 6:00-6:59, [7] = 9:00-9:59 etc.
  dh.fetch_prorail_data_from_path(file_path + '/' + fLst.name[hour])

  pd.set_option('float_format', '{:f}'.format)

  UTC0 = UTCs.at[0, 'UTC']
  UTC1 = UTCs.at[1, 'UTC']
  UTC2 = UTCs.at[2, 'UTC']
  UTC3 = UTCs.at[3, 'UTC']
  UTC4 = UTCs.at[4, 'UTC']
  UTC5 = UTCs.at[5, 'UTC']

  train0 = dh.df[dh.df['date_time_utc']== 	UTC0]
  train1 = dh.df[dh.df['date_time_utc']== 	UTC1]
  train2 = dh.df[dh.df['date_time_utc']== 	UTC2]
  train3 = dh.df[dh.df['date_time_utc']== 	UTC3]
  train4 = dh.df[dh.df['date_time_utc']== 	UTC4]
  train5 = dh.df[dh.df['date_time_utc']== 	UTC5]

  dens0.loc[i,1] = train0[train0['x_pos'] < rightbound_1]['tracked_object'].count() / area_1
  dens0.loc[i,2] = train0[(train0['x_pos'] > leftbound_2) & (train0['x_pos']< rightbound_2)]['tracked_object'].count() / area_2
  dens0.loc[i,3] = train0[(train0['x_pos'] > leftbound_3) & (train0['x_pos']< rightbound_3)]['tracked_object'].count() / area_3

  dens1.loc[i,1] = train0[train0['x_pos'] < rightbound_1]['tracked_object'].count() / area_1
  dens1.loc[i,2] = train0[(train0['x_pos'] > leftbound_2) & (train0['x_pos']< rightbound_2)]['tracked_object'].count() / area_2
  dens1.loc[i,3] = train0[(train0['x_pos'] > leftbound_3) & (train0['x_pos']< rightbound_3)]['tracked_object'].count() / area_3

  dens2.loc[i,1] = train0[train0['x_pos'] < rightbound_1]['tracked_object'].count() / area_1
  dens2.loc[i,2] = train0[(train0['x_pos'] > leftbound_2) & (train0['x_pos']< rightbound_2)]['tracked_object'].count() / area_2
  dens2.loc[i,3] = train0[(train0['x_pos'] > leftbound_3) & (train0['x_pos']< rightbound_3)]['tracked_object'].count() / area_3

  dens3.loc[i,1] = train0[train0['x_pos'] < rightbound_1]['tracked_object'].count() / area_1
  dens3.loc[i,2] = train0[(train0['x_pos'] > leftbound_2) & (train0['x_pos']< rightbound_2)]['tracked_object'].count() / area_2
  dens3.loc[i,3] = train0[(train0['x_pos'] > leftbound_3) & (train0['x_pos']< rightbound_3)]['tracked_object'].count() / area_3

  dens4.loc[i,1] = train0[train0['x_pos'] < rightbound_1]['tracked_object'].count() / area_1
  dens4.loc[i,2] = train0[(train0['x_pos'] > leftbound_2) & (train0['x_pos']< rightbound_2)]['tracked_object'].count() / area_2
  dens4.loc[i,3] = train0[(train0['x_pos'] > leftbound_3) & (train0['x_pos']< rightbound_3)]['tracked_object'].count() / area_3

  dens5.loc[i,1] = train0[train0['x_pos'] < rightbound_1]['tracked_object'].count() / area_1
  dens5.loc[i,2] = train0[(train0['x_pos'] > leftbound_2) & (train0['x_pos']< rightbound_2)]['tracked_object'].count() / area_2
  dens5.loc[i,3] = train0[(train0['x_pos'] > leftbound_3) & (train0['x_pos']< rightbound_3)]['tracked_object'].count() / area_3

  if i%2 == 0:
    UTCs['UTC'] = UTCs['UTC'] + 3600000
  else:
      UTCs['UTC'] = UTCs['UTC'] + 82800000
  
  i = i+1

14
targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220214/
Files listed. Accessible as <this-object>.filelist
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220214//EHV_Platform2.1_2022021407_trajectorie.parquet
data fetched. Accessible as <this-object>.df
14
targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220214/
Files listed. Accessible as <this-object>.filelist
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220214//EHV_Platform2.1_2022021408_trajectorie.parquet
data fetched. Accessible as <this-object>.df
15
targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220215/
Files listed. Accessible as <this-object>.filelist
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220215//EHV_Platform2.1_2022021507_trajectorie.parquet
data fetched. Accessible as <this-object>.df
15
targeting path /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20220215/


In [125]:
i = 0
day = 13
while i < 10:
  
  if i%2 == 0:
    day = day + 1
    print(day)
    hour = 7
  else:
      hour = 8
  i = i +1



14
15
16
17
18


In [18]:
dens4

,1,2,3
0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000


In [45]:
train0

,date_time_utc,tracked_object,x_pos,y_pos
2108,1644818460000.000000,2308228,-4411.000000,58453.000000
2109,1644818460000.000000,2308229,890.000000,30228.000000
2110,1644818460000.000000,2308231,2923.000000,11153.000000
2111,1644818460000.000000,2308233,-5271.000000,1930.000000


In [ ]:
w